In [ ]:
#TabulaRazr - specific to calculate  - TABLE Parser
#Infers a table with arbitrary number of columns from reoccuring patterns in text lines
#(c) Alexander Hirner 2016, no redistribution without permission
#Contributions: ____ (refactoring), UI styling (), ....


#Main assumptions Table identificatin:
#1) each row is either in one line or not a row at all
#2) each column features at least one number (=dollar amount)
#2a) each column features at least one date-like string [for time-series only]
#3) a table exists if rows are in narrow consecutive order and share similarities --> scoring algo [DONE] 
#4) each column is separated by more than x consecutive whitespace indicators (e.g. '  ' or '..')

#Feature List Todo:
#1) Acknowledge footnotes / make lower meta-data available
#2) make delimiter length smartly dependent on number of columns (possible iterative approach)
#3) improve captioning: expand non canonical values in tables [DONE] .. but not to the extent how types match up  --> use this to further
## delineate between caption and headers
#4) UI: parameterize extraction on the show page on the fly
#5) deeper type inference on token level: type complex [DONE], subtype header (centered, capitalized), 
## subtype page nr., type free flow [DONE, need paragraph]
#5a) re
#6) Respect negative values with potential '-' for numerical values
#7)
#8) classify tables with keywords (Muni Bonds) and unsupervised clustering (Hackathon)
#9) Restructure folder and URI around MD5 hash (http://stackoverflow.com/questions/24570066/calculate-md5-from-werkzeug-datastructures-filestorage-without-saving-the-object)
#10) proper logging
#11) include tesseract for OCR capabilities (quickstart guide: http://pythontips.com/2016/02/25/ocr-on-pdf-files-using-python/)

In [1]:
from __future__ import print_function

import os
import glob
import codecs
import json

import string

from backend import *
from data_query import *

UPLOAD_FOLDER = './static/ug'
PROJECT = 'muni_bonds_bulk'

path = os.path.join(UPLOAD_FOLDER, PROJECT, '*.tables.json')
table_files = glob.glob(path)

In [2]:
def clean_string(s):
    lc = s.encode('ascii', errors='ignore').lower()#.translate(remove_punctuation_map)
    return lc.translate(None, string.punctuation + '0123456789').strip()
    
from collections import Counter

table_counter = Counter()
terms_stripped = Counter()
terms_lc_cleaned = Counter()

tables_looked_at = 0
confidences = []
no_result_files = []
funny_tables = {}
funny_rows = {}
funny_values = ['NaN', 'Introduction', '']

filter_file = os.path.join('static', 'filters', 'funds.json')
with codecs.open(filter_file, "r", "utf-8", errors="replace") as file:
    _filter = json.load(file) 

print ("Processing with filter %s" % str(_filter))

Processing with filter {u'headers': {u'threshold': 0.35, u'terms': [u'USES OF FUNDS']}, u'name': u'Estimated use and sources of funds'}


In [ ]:



#Get all tables
for i,f in enumerate(table_files):

    with codecs.open(f, 'r', 'utf-8') as file:
        tables = json.load(file)
        tables_looked_at += len(tables)
        
        filename = f.split(r'/')[-1].replace('.tables.json', '')
        
        filter_results = []
        for t in filter_tables(tables.values(), _filter):
            if len(filter_results) == 0 or t[0] >= max(r[0] for r in filter_results):
                filter_results.append(t)
        
        table_counter[len(filter_results)] += 1        
        if len(filter_results):

            #Only keep first one
            confidence, table, _, _ = max( sorted( filter_results, key = lambda t: t[1]['begin_line'] ), 
                                          key = lambda t: t[0])
            confidences.append(confidence)
            if len(table['captions']) != 2 or table['subtypes'][1] != 'dollar':
                funny_tables[filename] = table['begin_line']
            for row in table['data']:
                #Prune for rows that don't have (the right) data
                #if True:
                if len(row) > 1 and 'subtype' in row[1] and row[1]['subtype'] == 'dollar':
                    first_term = row[0]['value'].strip()
                    if first_term in funny_values:
                        if filename in funny_rows: funny_rows[filename].append(row)
                        else: funny_rows[filename] = [row]
                        
                    terms_stripped[first_term] += 1
                    terms_lc_cleaned[clean_string(first_term)] += 1

                #It's probably an interims caption (or from the TOC!)   
                else:
                    if filename in funny_rows: funny_rows[filename].append(row)
                    else: funny_rows[filename] = [row]
        else:
            no_result_files.append(filename)
        
    if ( (i+1) % 100 ) == 0:
        print ("%i files and %i tables processed... with %i best matches and so far %i/%i unique terms" % \
               (i+1, tables_looked_at, len(confidences), len(terms_stripped), len(terms_lc_cleaned)))

    if i > 200: break


print(table_counter.most_common())
print(terms_lc_cleaned.most_common())
#print(no_result_files)
#print(funny_tables)
#print(funny_rows)

results = {'high_confidence_candidates' : table_counter.most_common(),
           'tables_looked_at' : tables_looked_at,
           'tables_canonical' : len(confidences),
           'confidence_mean' : sum(confidences) / len(confidences),
           'confidences' : confidences, 
           'unique_raw_terms' : len(terms_stripped),
           'unique_cleaned_terms' : len(terms_lc_cleaned),
           'raw_term_freq' : terms_stripped,
           'clean_term_freq' : terms_lc_cleaned,
           'no_table_files' : no_result_files,
           'funny_tables' : funny_tables,
           'funny_rows' : funny_rows
          }

In [ ]:
with codecs.open("funds_stats.results.json", "w", "utf-8") as file:
    json.dump(results, file)

In [ ]:
####

In [3]:
with codecs.open("funds_stats.results.json", "r", "utf-8") as file:
    results = json.load(file)

In [4]:
len(results["no_table_files"]) + len(results["confidences"])

6777

In [5]:
import xlwt

bold = xlwt.Style.easyxf("font: bold on")

def write_table(sheet, keys, values, row, c_offset = 0, column_style = bold):
    for j, k in enumerate(keys):
        sheet.write(row, c_offset+j, k, column_style)
    row += 1
    for v in values:
        for j, vv in enumerate(v):
            sheet.write(row, c_offset+j, vv)
        row +=1
    return row

In [26]:
url_prefix = "http://tabularazr.eastus.cloudapp.azure.com:7081/show/"+PROJECT+'/'

In [37]:
wkb = xlwt.Workbook(encoding='utf-8')
s_summary, s_raw_tf, s_clean_tf, s_confidence, s_no_table, s_funny_tables, s_funny_rows = \
    (wkb.add_sheet(s) for s in ['summary', 'raw_TF', 'clean_TF', 'confidence', 'no_table', 'funny_tables', 'funny_rows'])

In [38]:
i = 0
s_summary.write(i,0, 'Filter used', bold)
s_summary.write(i,1, str(_filter))
i+=2
s_summary.write(i,0, 'Distribution of good table matches per document', bold)
i+=1
i = write_table(s_summary, ['Nr. of Table Candidates', 'Nr. of Documents'], 
                results["high_confidence_candidates"], i)

i+=1
s_summary.write(i, 2, 'Total nr. of Table Candidates')
s_summary.write(i, 3, 'out of..')
i+=1
s_summary.write(i, 2, results['tables_canonical'])
s_summary.write(i, 3, results['tables_looked_at'])

i = write_table(s_confidence, ['Confidence in best Table found'], ([c] for c in results['confidences']), 0)
i = write_table(s_no_table, ['Files with no suitable table found', 'URL'], 
                ( ([c], url_prefix+c) for c in results['no_table_files'] ), 0)

i = write_table(s_raw_tf, ['Term (raw)', 'Frequency'], 
                (tf for tf in Counter(results['raw_term_freq']).most_common() ), 0)
i = write_table(s_clean_tf, ['Term (cleaned)', 'Frequency'], 
                (tf for tf in Counter(results['clean_term_freq']).most_common() ), 0)

s_funny_tables.write(0,4, "[as returned by filter but with <> 2 rows, and/or no $ value in the 2nd column]")
i = write_table(s_funny_tables, ['Funny Tables in File', 'Table ID',  'URL'], 
                ( (f, t, url_prefix+f+'#'+str(t)) for f, t in results['funny_tables'].iteritems() ), 0)

In [39]:
wkb.save('fund_filter_results.xls')

In [36]:
for f, t in results['funny_tables'].iteritems():
    print (f, t)

089333LG1-IRIS-cb961.pdf.txt 335
798458GJ8-IRIS-zd272.pdf.txt 548
121653JZ3-IRIS-cs749.pdf.txt 650
968206RW4-IRIS-cm835.pdf.txt 1577
516840VE2-IRIS-dg649.pdf.txt 4776
545446BB1-IRIS-cl578.pdf.txt 1555
357172VV4-IRIS-da434.pdf.txt 5117
92817TCN4-IRIS-cj924.pdf.txt 3115
76541VLG0-IRIS-ef860.pdf.txt 529
544648QS0-IRIS-y1233.pdf.txt 3944
878164DC3-IRIS-dn836.pdf.txt 4035
048483ZN4-IRIS-dm688.pdf.txt 2000
514462E63-IRIS-ed442.pdf.txt 886
429326UY5-IRIS-dj698.pdf.txt 5652
801365BF3-IRIS-bd681.pdf.txt 1674
592795JM1-IRIS-ec340.pdf.txt 864
928172D36-IRIS-dp776.pdf.txt 6342
846718LU6-IRIS-dr487.pdf.txt 3286
928109E60-IRIS-cs029.pdf.txt 12782
496782MH3-IRIS-cq807.pdf.txt 863
187145BK2-IRIS-cq722.pdf.txt 461
447819DT7-IRIS-ea384.pdf.txt 586
92817SL22-IRIS-cx722.pdf.txt 5298
074437CC6-IRIS-ak970.pdf.txt 927
845040JV7-IRIS-ct875.pdf.txt 975
585483KT6-IRIS-dn228.pdf.txt 5577
8827224L2-IRIS-ck570.pdf.txt 462
16532LHN9-IRIS-cd737.pdf.txt 486
880162CF7-IRIS-di946.pdf.txt 6233
92812VCY0-IRIS-dy557.pdf.t

In [23]:
results['funny_tables']

{u'004014EX0-IRIS-ce085.pdf.txt': 4545,
 u'004014FQ4-IRIS-dc631.pdf.txt': 4264,
 u'006644A24-IRIS-db481.pdf.txt': 5462,
 u'006644A81-IRIS-di476.pdf.txt': 5291,
 u'006644A81-IRIS-di496.pdf.txt': 5291,
 u'006644D47-IRIS-di494.pdf.txt': 5291,
 u'006644VC9-IRIS-ck643.pdf.txt': 4880,
 u'011338TD2-IRIS-dg925.pdf.txt': 6294,
 u'011411CT0-IRIS-cq831.pdf.txt': 3748,
 u'011411DJ1-IRIS-cp409.pdf.txt': 3698,
 u'015289BL4-IRIS-co186.pdf.txt': 5525,
 u'015302M33-IRIS-cd586.pdf.txt': 8091,
 u'015302N65-IRIS-dv569.pdf.txt': 8252,
 u'015302R38-IRIS-dv569.pdf.txt': 8252,
 u'015302S45-IRIS-db454.pdf.txt': 8369,
 u'016721DQ2-IRIS-dt413.pdf.txt': 1466,
 u'016721EL2-IRIS-dt413.pdf.txt': 1466,
 u'020768DE4-IRIS-dm014.pdf.txt': 1134,
 u'022357VH7-IRIS-dc280.pdf.txt': 5021,
 u'022429VK7-IRIS-dy054.pdf.txt': 5921,
 u'022447WP7-IRIS-ed040.pdf.txt': 6868,
 u'022447WS1-IRIS-dh072.pdf.txt': 6551,
 u'023015UZ1-IRIS-dh414.pdf.txt': 6485,
 u'023015VF4-IRIS-dj457.pdf.txt': 5989,
 u'04144RCJ6-IRIS-cz225.pdf.txt': 8507,


In [ ]:
from fuzzywuzzy import fuzz
#print (fuzz.partial_ratio("this is a test!", "this text is"))
#print (fuzz.partial_ratio("USES OF FUNDS", "USES from FUNDS"))
#print (fuzzy_str_match("aaaaaaaaaa text this", "this test is"))

print (fuzzy_str_match("USES OF FUNDS", "USES OF FUNDS"))
print (fuzzy_str_match("USES OF FUNDS", "USES of FUNDS"))
print (fuzzy_str_match("USES OF FUNDS", "uses of funds"))
print (fuzzy_str_match("USES OF FUNDS", "USES which include other FUNDS"))


print (fuzzy_str_match("USES OF FUNDS", "Note 5 - Assets Limited as to Use"))
print (fuzzy_str_match("USES OF FUNDS", "FINANCIAL STATEMENT AMOUNTS - Continued"))

print (fuzzy_str_match("USES OF FUNDS", "FUNDS OF USES"))


print (fuzzy_str_match("USES OF FUNDS", "TABLE OF CONTENTS"))

print (fuzzy_str_match("Debt Service", "Total Periodic Debt Service"))

#print (fuzzy_str_match("USES OF FUNDS", "Note 5 - Assets Limited as to Use"))
#print (fuzzy_str_match("USES OF FUNDS", "FINANCIAL STATEMENT AMOUNTS - Continued"))


In [ ]:
print (fuzzy_str_match("Maturity", "ii"))
#print (fuzz.partial_ratio("Maturity", "i"))
print (fuzzy_str_match("Maturity", "Matarit"))
print (fuzzy_str_match("Maturity", "i"))
print (fuzzy_str_match("Maturity", "     i   "))


print ("Maturity".lower() in "i".lower())